In [16]:
#shiraz yeshayahu 
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

destination_data = pd.read_csv("C:/Users/WIN10/OneDrive/שולחן העבודה/WIN10/שנה ג סמסטר ב/data science projects/Assignments/destinations_LP_crawler_Ex5.csv",encoding='latin-1')
display(destination_data)

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


In [17]:
LP_destinations = pd.DataFrame(columns= ["City", "Country", "Description"] )
LP_destinations["City"] = destination_data["city"]
display(LP_destinations)

,City,Country,Description
0,Antalya,NaN,NaN
1,Athens,NaN,NaN
2,Ayia-napa,NaN,NaN
3,Bangkok,NaN,NaN
4,Barcelona,NaN,NaN
...,...,...,...
71,Venice,NaN,NaN
72,Vienna,NaN,NaN
73,Warsaw,NaN,NaN
74,Zakynthos,NaN,NaN


In [18]:
for city in range(len(LP_destinations)):
    if '¿' in LP_destinations.loc[city,'City'] or 'ï' in LP_destinations.loc[city,'City']:
        LP_destinations.loc[city,'City']=LP_destinations.loc[city,'City'].replace('ï', 'o')
        LP_destinations.loc[city,'City']=LP_destinations.loc[city,'City'].replace('¿', 'w')


In [19]:
country_list = list()
Description_list = list()


for city in range(len(LP_destinations)): 

    address = LP_destinations["City"].loc[city]
    api_key = "your a"   
    url_geocode  = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (address, api_key)

    try:
        response_geocode = requests.get(url_geocode)

        if not  (response_geocode.status_code == 200):
            print("HTTP error", response.status_code)
            
        else:
            response_geocode = requests.get(url_geocode).json()
            country = response_geocode["results"][0]["formatted_address"].split(",")[-1].strip()
            for num in country:
                if num.isdigit():
                    country = response_geocode["results"][0]["formatted_address"].split(",")[-2].strip()
            country_list.append(country)
            
    except:
        print("somting went wrong with requests.get or response not in valid JSON format")

    try:
        city_LP = destination_data["city_LP"].iloc[city]
        url_Description = "https://www.lonelyplanet.com/%s/%s" % (country, city_LP)
        response_Description = requests.get(url_Description)
        soup = BeautifulSoup(response_Description.content, 'html.parser')
        tag_description = soup.find("section", { "id" : "introduction" })
        description = tag_description.find("p").get_text()
        Description_list.append(description)
    except:
        print("somthing was wrong")

        
LP_destinations["Description"] = Description_list
LP_destinations["Country"] = country_list                

print(LP_destinations)
 

         City      Country                                        Description
0     Antalya       Turkey  Once seen simply as the gateway to the Turkish...
1      Athens       Greece  With equal measures of grunge and grace, Athen...
2   Ayia-napa       Cyprus  Endless blue-sky days and a coastline riddled ...
3     Bangkok     Thailand  Same same, but different. This Thailish T-shir...
4   Barcelona        Spain  Barcelona is an enchanting seaside city with b...
..        ...          ...                                                ...
71     Venice        Italy  Imagine the audacity of building a city of mar...
72     Vienna      Austria  Baroque streetscapes and imperial palaces set ...
73     Warsaw       Poland  A phoenix arisen from the ashes, Poland's capi...
74  Zakynthos       Greece  Zakynthos, also known by its Italian name Zant...
75     Zurich  Switzerland  Culturally vibrant, efficiently run and attrac...

[76 rows x 3 columns]


In [20]:
LP_destinations.to_csv('LP_destinations.csv', index=False)